# Using rail_fsps to generate galaxy rest-frame spectral energy distributions

author: Luca Tortorelli, Eric Charles, Sam Schmidt, Joe Zunzt and others

last run successfully: September 19th, 2022

This notebook demonstrates how to use RAIL to generate galaxy rest-frame spectral energy distributions (SEDs) with FSPS.

In order to generate SEDs, the user needs to provide as input a set of galaxy properties, namely galaxy ages, stellar metallicities, stellar velocity dispersions, gas ionizations, gas metallicities, e-folding times for parametric star-formation histories, fraction of mass formed in instantaneous burst, age at which the instantaneous burst occur, extinction expressed as E(B-V), fraction of galaxy luminosity due to AGN and AGN optical depths.

In this notebook, we will learn how to generate rest-frame SED from a RAIL Engine object. We will also show how to use Ceci to do the same, as well as how to run it in parallel using MPI.


In [ ]:
import os
from rail.core.utils import RAILDIR
import tables_io
from rail.core.stage import RailStage
from rail.creation.sed_generation import FSPSSedGenerator
import ceci

rail_fsps accepts as input a fits table with columns containing the galaxy properties. The table is read via tables_io. An example table is in the rail/examples_data/testdata directory. 

In [ ]:
trainFile = os.path.join(RAILDIR, 'rail', 'examples_data', 'testdata', 'test_fsps_sed.fits')
traindata_io = tables_io.read(trainFile)

We'll start by setting up the Rail data store. RAIL uses ceci, which is designed for pipelines rather than interactive notebooks, the data store will work around that and enable us to use data interactively. See the rail/examples/goldenspike_examples/goldenspike.ipynb example notebook for more details on the Data Store.

In [ ]:
DS = RailStage.data_store
DS.__class__.allow_overwrite = True

Let's create an FSPS Sed Generator. The latter has a number of configuration parameters to set. All the parameters have default values. Therefore, if the user is not sure about a particular value for a certain parameter, the latter can be left at default. The parameter names are the same as the ones used in the Python bindings of FSPS. A short description of each parameter can be found in rail/creation/sed_generation/sed_generator.py. 

The parameters that do not have correspondance with FSPS are:
- metal_name, which is the column name in the input fits file containing the values of the stellar metallicity in units of $\log(Z / Z_{\odot})$. If the parameter sfh_type=3, then a tabulated star-formation history file must be provided, which contains in itself values of the galaxy metallicity as function of time. In this case, the metallicities values in the fits table are ignored.
- agename, which is the column name in the input fits file containing the values of the stellar age in units of Gyr at which the rest-frame galaxy spectrum is computed.
- veldispname, which is the column name in the input fits file containing the values of the stellar velocity dispersions. The units can be either km/s or Angstrom, depending on the value given to the smooth_velocity parameter.
- gasloguname, which is the column name in the input fits file containing the values of the logarithm of the gas ionization parameter.
- gaslogzname, which is the column name in the input fits file containing the values of the logarithm of the gas-phase metallicity (see note in https://dfm.io/python-fsps/current/stellarpop_api/ regarding the values of stellar and gas-phase metallicity).
- tauname, which is the column name in the input fits file containing the values of the e-folding time of the star-formation history. This column values are ignored if sfh_type=0 or sfh_type=3.
- fburstname, which is the column name in the input fits file containing the values of the mass fraction formed in an instantaneous burst of star-formation. This column values are used if sfh_type=0 or sfh_type=4.
- tburstname, which is the column name in the input fits file containing the values of the Universe age in Gyr when the burst of star-formation occurred.
- ebvname, which is the column name in the input fits file containing the values of attenuation E(B-V).
- fagnname, which is the column name in the input fits file containing the values of fraction bolometric luminosity due to AGN.
- agntauname, which is the column name in the input fits file containing the values of optical depth of AGN dust torus.
- physical_units, which can be either True or False to output the rest-frame spectra in units of $erg\ s^{-1} \AA^{-1}$ or $L_{\odot} \AA^{-1}$, respectively.
- tabulated_sfh_file, which is the path to a text file containing a non-parametric star-formation history. The text file must have two mandatory columns and a third optional one: time since the beginning of the universe in Gyr, star-formation rate in units of $M_{\odot}\ yr^{-1}$ and the metallicity at each age, in units of absolute metallicity (e.g. Z=0.019 for Padova isochrones).
- tabulated_lsf_file, which is the path to a text file containing a wavelength dependent Gaussian line-spread function that will be applied to the SSPs. Its effect is going to be applied if and only if both smooth_lsf and smooth_velocity are True. The text file must have two mandatory columns and two optional ones: the wavelength in angstroms, the dispersion of the Gaussian LSF at the wavelengths given by the previous column, in km/s, the minimum and the maximum wavelengths (in $\AA$) for which smoothing will be applied.

In [ ]:
sed_generation_test = FSPSSedGenerator.make_stage(name='sed_generator_test', compute_vega_mags=False,
                                                  vactoair_flag=False, zcontinuous=1, add_neb_emission=False,
                                                  add_neb_continuum=False, add_stellar_remnants=True,
                                                  compute_light_ages=False, nebemlineinspec=False,
                                                  smooth_velocity=True, smooth_lsf=False, imf_type=1,
                                                  min_wavelength=4000, max_wavelength=7000, sfh_type=0,
                                                  dust_type=2, metal_name='logzsol', agename='tage', 
                                                  veldispname='sigma_smooth', gasloguname='gas_logu',
                                                  gaslogzname='gas_logz', tauname='tau', fburstname='fburst',
                                                  tburstname='tburst', ebvname='dust2', fagnname='fagn',
                                                  agntauname='agn_tau', physical_units=False, tabulated_sfh_file=None,
                                                  tabulated_lsf_file=None)

After setting up the parameters, we add the input fits data to the RAIL stage.

In [ ]:
sed_generation_test.add_data('input',traindata_io[0:1])

And then we run the FSPSSedGenerator in sequential mode. Note that each galaxy spectrum takes a few seconds to generate, so it is advisable to proceed in this way only for a limited sample of objects or for testing the code.

In [ ]:
sed_generation_test.run()

The next line shows how to store the output table in a variable. The output table is still in fits format. Besides containing a copy of the input galaxy properties, it also contains two additional columns: wavelength and spectrum, which contains rest-frame wavelengths in $\AA$ and rest-frame spectrum in specified units (see physical_units parameter).

In [ ]:
out_table = sed_generation_test.get_data('output')

The RAIL stages can be chained together conveniently using Ceci. The following is an example of how the FSPSSedGenerator can be run as one of the pipeline Ceci stages. 

In [ ]:
DS = RailStage.data_store
DS.__class__.allow_overwrite = True

In [ ]:
sed_generation_test = FSPSSedGenerator.make_stage(name='sed_generator_test', compute_vega_mags=False,
                                                  vactoair_flag=False, zcontinuous=1, add_neb_emission=False,
                                                  add_neb_continuum=False, add_stellar_remnants=True,
                                                  compute_light_ages=False, nebemlineinspec=False,
                                                  smooth_velocity=True, smooth_lsf=False, imf_type=1,
                                                  min_wavelength=4000, max_wavelength=7000, sfh_type=0,
                                                  dust_type=2, metal_name='logzsol', agename='tage', 
                                                  veldispname='sigma_smooth', gasloguname='gas_logu',
                                                  gaslogzname='gas_logz', tauname='tau', fburstname='fburst',
                                                  tburstname='tburst', ebvname='dust2', fagnname='fagn',
                                                  agntauname='agn_tau', physical_units=False, tabulated_sfh_file=None,
                                                  tabulated_lsf_file=None)

In [ ]:
pipe = ceci.Pipeline.interactive()
stages = [sed_generation_test]
for stage in stages:
    pipe.add_stage(stage)

In [ ]:
pipe.initialize(dict(input=trainFile), dict(output_dir='./temp_output', log_dir='./logs', resume=False, 
                                            nprocess=10), None)
pipe.save('./temp_output/pipe_saved.yml')
pr = ceci.Pipeline.read('./temp_output/pipe_saved.yml')
pr.run()

The creation of galaxy SEDs is a computationally intensive process. To speed things up, it is convenient to parallelize the process using MPI. To do that, one needs to set the parameters into the configuration file pipe_saved_config.yml. An example command to be run in the command line with n_cores is:

mpiexec -n n_cores --mpi python3 -m rail SedGenerator --input=test_fsps_sed.fits --name=sed_generator_test --config=pipe_saved_config.yml --output=output_sed_generator_test.fits
